In [1]:
!pip install -q git+https://github.com/openai/CLIP.git

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 2.3 MB/s eta 0:00:00


In [2]:
pip install -q deeplake

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.7/617.7 kB 30.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 102.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 7.7 MB/s eta

In [3]:
import clip
import deeplake
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [4]:
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [5]:
# Hyperparameters
batch_size = 5
num_epochs = 3

In [26]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)
model.to(device)

cuda


CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [8]:
train_ds = deeplake.load("hub://activeloop/pacs-train")
test_ds = deeplake.load("hub://activeloop/pacs-test")

train_loader = train_ds.pytorch(num_workers=0, batch_size = batch_size, shuffle = False)
test_loader = test_ds.pytorch(num_workers=0, batch_size = batch_size, shuffle = False)

\

Opening dataset in read-only mode as you don't have write permissions.


\

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/pacs-train



\

...

-

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/pacs-test



-

hub://activeloop/pacs-test loaded successfully.



/usr/local/lib/python3.10/dist-packages/deeplake/integrations/pytorch/common.py:137: UserWarning: Decode method for tensors ['images'] is defaulting to numpy. Please consider specifying a decode_method in .pytorch() that maximizes the data preprocessing speed based on your transformation.
  warnings.warn(


In [18]:
train_ds.visualize()
test_ds.visualize()

HINT: Please forward the port - 51229 to your local machine, if you are running on the cloud.
 * Serving Flask app 'dataset_visualizer'
 * Debug mode: off


In [17]:
class_names = ['dog', 'elephant', 'giraffe', 'guitar', 'horse', 'house', 'person']

In [27]:
##### CLIP Zero-Shot Classification for TRAIN Data of PACS #####
correct_pred_train = 0
total_pred_train = 0
tokenized_text_desp = clip.tokenize(class_names).to(device)
for images, labels,_,_ in tqdm(train_loader):
    images = images.reshape(-1, 3, 227, 227).float()
    labels = labels.squeeze()
    images_input = images.to(device)

    with torch.no_grad():
        image_features = model.encode_image(images_input).float()
        text_features = model.encode_text(tokenized_text_desp).float()

    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = text_features @ image_features.T
    preds = np.argmax(similarity.cpu().numpy(), axis=0)
    correct_pred_train += (preds == labels).sum()
    total_pred_train += len(labels)
print(f'Train Accuracy: {100*correct_pred_train / total_pred_train:.4f}')

100%|██████████| 1796/1796 [00:48<00:00, 37.37it/s]

Train Accuracy: 14.4703


In [28]:
##### CLIP Zero-Shot Classification for TEST Data of PACS #####
correct_pred = 0
total_pred = 0
tokenized_text_desp = clip.tokenize(class_names).to(device)
for images, labels,_,_ in tqdm(test_loader):
    images = images.reshape(-1, 3, 227, 227).float()
    labels = labels.view(-1)
    images_input = images.to(device)
    with torch.no_grad():
        image_features = model.encode_image(images_input).float()
        text_features = model.encode_text(tokenized_text_desp).float()

    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = text_features @ image_features.T
    preds = np.argmax(similarity.cpu().numpy(), axis=0)
    correct_pred += (preds == labels).sum()
    total_pred += labels.size(0)
print(f'Test Accuracy: {100*correct_pred / total_pred:.4f}')

100%|██████████| 1999/1999 [00:53<00:00, 37.08it/s]

Test Accuracy: 14.4730
